# Development notebook for optical evaporative cooling DSMC
Patrick Gleeson, Semester 2 2021

In [1]:
# Simulation parameters
const dt = 0.1 # timestep
const m = 1e-27 # atom mass
const N0 = 10000 # initial number of atoms

10000

In [15]:
# Set random generator seed
using Random
Random.seed!(1)

## Method 1: Atom struct

In [2]:
# Represent the position and momentum of an atom
mutable struct Atom
    r::Vector{Float64} # position
    v::Vector{Float64} # velocity
end


In [3]:
# Initialise atoms in unit cube with random velocities
atoms = Vector{Atom}() # atom storage
for i in 1:N0
    push!(atoms, Atom(rand(3), rand(3)))
end

In [4]:
# Plot trajectories

# using Pkg; Pkg.add("Plots")
using Plots

function plotAtoms(atoms)
    x_values = [atom.r[1] for atom in atoms]
    y_values = [atom.r[2] for atom in atoms]
    scatter(x_values, y_values, title = "z-projection")
end


plotAtoms (generic function with 1 method)

In [5]:
# Perform collisionless evolution with no potential
function step!(atoms, dt)
    for atom in atoms
        atom.r += atom.v * dt
    end

    return atoms
end

step! (generic function with 1 method)

In [6]:
# Free evolution for certain time
function freeEvolve!(atoms, t_max, dt)
    t = 0
    while t < t_max
        step!(atoms, dt)
        t += dt
    end
end

freeEvolve! (generic function with 1 method)

## Method 2: 2D Array

In [7]:
# Initialise random positions and velocities in unit cube
positions = rand(3, N0) :: Matrix{Float64}
velocities = rand(3, N0) :: Matrix{Float64}

3×10000 Matrix{Float64}:
 0.822958  0.836126   0.740351   0.584458   …  0.0460656  0.410584  0.654696
 0.239832  0.0566834  0.0326669  0.0617902     0.383997   0.289975  0.268674
 0.891546  0.394944   0.298327   0.0690056     0.758958   0.500926  0.715779

In [8]:
# Stepping is:
function arraystep!(positions, velocities, dt)
    positions .+= velocities * dt
end

arraystep! (generic function with 1 method)

In [9]:
# Free evolve
function arrayFreeEvolve!(positions, velocities, t_max, dt)
    t = 0
    while t < t_max
        arraystep!(positions, velocities, dt)
        t += dt
    end
end

arrayFreeEvolve! (generic function with 1 method)

## Comparison

In [10]:
# using Pkg
# Pkg.add("BenchmarkTools")
using BenchmarkTools
t_max = 10

10

In [11]:
@benchmark arrayFreeEvolve!(positions, velocities, t_max, dt)

BenchmarkTools.Trial: 139 samples with 1 evaluation.
 Range (min … max):  27.799 ms … 131.621 ms  ┊ GC (min … max):  0.00% … 30.68%
 Time  (median):     35.178 ms               ┊ GC (median):    17.76%
 Time  (mean ± σ):   36.052 ms ±  11.553 ms  ┊ GC (mean ± σ):  11.73% ± 10.25%

   ▄▃▁  █▃                                                      
  ▆███▄▇██▇▅▃▁▃▃▃▁▁▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  27.8 ms         Histogram: frequency by time          102 ms <

 Memory estimate: 23.12 MiB, allocs estimate: 202.

In [12]:
@benchmark freeEvolve!(atoms,t_max,dt)

BenchmarkTools.Trial: 22 samples with 1 evaluation.
 Range (min … max):  216.669 ms … 242.865 ms  ┊ GC (min … max): 10.23% … 9.24%
 Time  (median):     226.479 ms               ┊ GC (median):    10.04%
 Time  (mean ± σ):   227.866 ms ±   7.028 ms  ┊ GC (mean ± σ):   9.97% ± 0.86%

  ▁    ▁  ▁▁█    ▁  ▁ ▁█  ▁ ▁ ▁▁     ▁  ▁  ▁▁      ▁  ▁       ▁  
  █▁▁▁▁█▁▁███▁▁▁▁█▁▁█▁██▁▁█▁█▁██▁▁▁▁▁█▁▁█▁▁██▁▁▁▁▁▁█▁▁█▁▁▁▁▁▁▁█ ▁
  217 ms           Histogram: frequency by time          243 ms <

 Memory estimate: 215.76 MiB, allocs estimate: 2020000.

## Visualisation (struct method)

In [13]:
#=
# Run for 10 virtual seconds and create gif.
t = 0
t_max = 10

anim = @animate for i in 1:ceil(t_max/dt)
    plotAtoms(step!(atoms, dt))
end

gif(anim, "anim_fps15.gif", fps = 15)
=#